# Function Generation for the Training of λ-Nets

## Package installation (uncommand first line to install packages at the beginning)

In [1]:
%%script false --no-raise-error
#if some errors occur during the installation consider using "sudo pip install"
!pip install numpy
!pip install pandas
!pip install truth-table-generator
!pip install more-itertools
!pip install tqdm
!pip install joblib
!pip install scipy
!pip install PrettyTable
!pip install colored
!pip install scikit-learn
!pip install keras
!pip install ipython
!pip install livelossplot
!pip install matplotlib
!pip install seaborn
!pip install tensorflow
!pip install tensorflow-gpu

## Specitication of Experiment Settings

In [2]:
import math
def nCr(n,r):
    f = math.factorial
    return f(n) // f(r) // f(n-r)

In [3]:
d = 3  
n = 4
sparsity = nCr(n+d, d)


x_max = 1 
x_min = -1
x_step = 0.01
a_max = 10 
a_min = -10
a_step = 0.001

n_jobs = -3

lambda_dataset_size = 1000 #specify the number of data points to calculate the function values for (determines the lambda net training size)

interpretation_dataset_size = 100000 #specifies the number of functions generated (specifies the interpretation-net dataset size)

same_training_all_lambda_nets = False


In [4]:
##############DO NOT CHANGE###################
variables = 'abcdefghijklmnopqrstuvwxyz'[:n]

RANDOM_SEED = 42

if same_training_all_lambda_nets:
    training_string = '_same'
else:
    training_string = '_diverse'
    
print('Variables: ' + str(n) + ' (' + variables + ')')
print('Degree: ' + str(d))
print('Sparsity: ' + str(sparsity)) 
print('Lambda-Net Dataset Size: ' + str(lambda_dataset_size))
print('I-Net Dataset Size: ' + str(interpretation_dataset_size))
      
print('Coefficient Range: ' + '[' + str(a_min) + ', ' + str(a_max) + ']')
print('Variable Range: ' + '[' + str(x_min) + ', ' + str(x_max) + ']')

Variables: 4 (abcd)
Degree: 3
Sparsity: 35
Lambda-Net Dataset Size: 1000
I-Net Dataset Size: 100000
Coefficient Range: [-10, 10]
Variable Range: [-1, 1]


## Imports

In [5]:
import ttg
from itertools import product       # forms cartesian products
from more_itertools import random_product 
from tqdm import tqdm_notebook as tqdm
import pickle

import numpy as np

import pandas as pd
from joblib import Parallel, delayed

import random 
from random import sample 
random.seed(42)

import os

directory_names = ['parameters', 'plotting', 'saved_polynomial_lists', 'results', 'saved_models', 'weights', 'weights_training']
if not os.path.exists('./data'):
    os.mkdir('./data')
for directory_name in directory_names:
    path = './data/' + directory_name
    if not os.path.exists(path):
        os.mkdir(path)
        
import seaborn as sns
sns.set_style("darkgrid")
        
import random 
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)




## Utility functions

In [6]:
#test for exact equality
def arreq_in_list(myarr, list_arrays):
    return next((True for elem in list_arrays if np.array_equal(elem, myarr)), False)

In [7]:
ALPHABET = \
  "0123456789abcdefghijklmnopqrstuvwxyz"

def encode (n):
    try:
        return ALPHABET [n]
    except IndexError:
        raise Exception ("cannot encode: %s" % n)

def dec_to_base (dec = 0, base = 16):
    if dec < base:
        return encode (dec)
    else:
        return dec_to_base (dec // base, base) + encode (dec % base)

# Function Generation

In [8]:
list_of_monomial_identifiers_extended = []
for i in tqdm(range((d+1)**n)):    
    monomial_identifier = dec_to_base(i, base = (d+1)).zfill(n) 
    list_of_monomial_identifiers_extended.append(monomial_identifier)

print('List length: ' + str(len(list_of_monomial_identifiers_extended)))
print('Number of monomials in a polynomial with ' + str(n) + ' variables and degree ' + str(d) + ': ' + str(nCr(n+d, d)))
print('Sparsity: ' + str(sparsity))
print(list_of_monomial_identifiers_extended)

list_of_monomial_identifiers = []
for monomial_identifier in tqdm(list_of_monomial_identifiers_extended):
    monomial_identifier_values = list(map(int, list(monomial_identifier)))
    if sum(monomial_identifier_values) <= d:
        list_of_monomial_identifiers.append(monomial_identifier)

print('List length: ' + str(len(list_of_monomial_identifiers)))
print('Number of monomials in a polynomial with ' + str(n) + ' variables and degree ' + str(d) + ': ' + str(nCr(n+d, d)))
print('Sparsity: ' + str(sparsity))
print(list_of_monomial_identifiers)



List length: 256
Number of monomials in a polynomial with 4 variables and degree 3: 35
Sparsity: 35
['0000', '0001', '0002', '0003', '0010', '0011', '0012', '0013', '0020', '0021', '0022', '0023', '0030', '0031', '0032', '0033', '0100', '0101', '0102', '0103', '0110', '0111', '0112', '0113', '0120', '0121', '0122', '0123', '0130', '0131', '0132', '0133', '0200', '0201', '0202', '0203', '0210', '0211', '0212', '0213', '0220', '0221', '0222', '0223', '0230', '0231', '0232', '0233', '0300', '0301', '0302', '0303', '0310', '0311', '0312', '0313', '0320', '0321', '0322', '0323', '0330', '0331', '0332', '0333', '1000', '1001', '1002', '1003', '1010', '1011', '1012', '1013', '1020', '1021', '1022', '1023', '1030', '1031', '1032', '1033', '1100', '1101', '1102', '1103', '1110', '1111', '1112', '1113', '1120', '1121', '1122', '1123', '1130', '1131', '1132', '1133', '1200', '1201', '1202', '1203', '1210', '1211', '1212', '1213', '1220', '1221', '1222', '1223', '1230', '1231', '1232', '1233', '1


List length: 35
Number of monomials in a polynomial with 4 variables and degree 3: 35
Sparsity: 35
['0000', '0001', '0002', '0003', '0010', '0011', '0012', '0020', '0021', '0030', '0100', '0101', '0102', '0110', '0111', '0120', '0200', '0201', '0210', '0300', '1000', '1001', '1002', '1010', '1011', '1020', '1100', '1101', '1110', '1200', '2000', '2001', '2010', '2100', '3000']


In [9]:
list_of_polynomials = []

if interpretation_dataset_size/((a_max-a_min)*10**int(-np.log10(a_step)))**(nCr(n+d, d)) <= 10e-4:
    while len(list_of_polynomials) < interpretation_dataset_size:
        random_polynomial = list(random_product([i*a_step for i in range(int(a_min*10**int(-np.log10(a_step))), int(a_max*10**int(-np.log10(a_step))))], repeat=nCr(n+d, d)))
        if random_polynomial not in list_of_polynomials:
            list_of_polynomials.append(random_polynomial)
else:
    all_polynomials_list = list(product([i*a_step for i in range(int(a_min*10**int(-np.log10(a_step))), int(a_max*10**int(-np.log10(a_step))))], repeat=nCr(n+d, d)))
    list_of_polynomials = [all_polynomials_list[i] for i in np.random.choice(len(all_polynomials_list), interpretation_dataset_size, replace=False)]
    del all_polynomials_list

for polynomial in tqdm(list_of_polynomials):
    sparsity_indices = np.random.choice(nCr(n+d, d), nCr(n+d, d)-sparsity, replace=False)
    for sparsity_index in sparsity_indices:                            
        polynomial[sparsity_index] = 0
                                
polynomials_list_df = pd.DataFrame(data=list_of_polynomials, columns=list_of_monomial_identifiers)

    
print(len(list_of_monomial_identifiers))
print(polynomials_list_df.shape)


35
(100000, 35)


In [10]:
polynomials_list_df.head()

,0000,0001,0002,0003,0010,0011,0012,0020,0021,0030,...,1020,1100,1101,1110,1200,2000,2001,2010,2100,3000
0,-6.352,-9.181,-0.988,-1.976,-2.686,-5.428,-6.642,7.870,-7.152,9.349,...,9.309,-0.885,-9.788,-4.769,3.848,1.149,-0.895,-4.906,-2.945,1.029
1,-6.651,-6.961,2.449,-6.831,1.763,1.270,9.782,-1.333,-8.577,5.054,...,-6.691,2.455,-0.892,4.857,1.954,-4.671,2.130,1.641,-3.135,-1.252
2,-7.661,9.960,-4.393,7.502,-1.979,-4.646,5.147,2.433,-1.155,8.250,...,5.035,-5.319,-1.321,-5.425,-1.919,8.394,7.661,-1.391,9.155,4.038
3,9.121,3.087,1.861,-2.814,-5.468,6.696,6.171,-7.022,-8.457,-6.408,...,-1.257,1.146,-6.345,-0.383,4.246,-4.818,4.867,-9.894,-1.370,6.403
4,-4.146,6.635,-6.514,-0.221,6.635,9.954,-3.483,-4.992,2.252,-4.707,...,-7.194,5.924,-7.733,7.455,-5.880,-5.793,5.574,8.015,-4.590,-1.315


In [11]:
def generate_random_x_values(size, x_max, x_min, x_step, numnber_of_variables):
    x_values_list = []
    
    for j in range(size):
        values = np.round(np.array(random_product(np.arange(x_min, x_max, x_step), repeat=numnber_of_variables)), int(-np.log10(x_step)))
        while arreq_in_list(values, x_values_list):
                values = np.round(np.array(random_product(np.arange(x_min, x_max, x_step), repeat=numnber_of_variables)), int(-np.log10(x_step)))         
        x_values_list.append(values)
    
    return x_values_list


In [12]:
if same_training_all_lambda_nets:
    x_values = generate_random_x_values(lambda_dataset_size, x_max, x_min, x_step, n)
    x_values_list = [x_values for _ in range(interpretation_dataset_size)]
else:
    parallel = Parallel(n_jobs=n_jobs, verbose=3, backend='loky')
    x_values_list = parallel(delayed(generate_random_x_values)(lambda_dataset_size, x_max, x_min, x_step, n) for _ in range(interpretation_dataset_size))
    del parallel


[Parallel(n_jobs=-3)]: Using backend LokyBackend with 22 concurrent workers.
[Parallel(n_jobs=-3)]: Done  84 tasks      | elapsed:   43.8s
[Parallel(n_jobs=-3)]: Done 244 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-3)]: Done 468 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-3)]: Done 756 tasks      | elapsed:  5.9min
[Parallel(n_jobs=-3)]: Done 1108 tasks      | elapsed:  8.6min
[Parallel(n_jobs=-3)]: Done 1524 tasks      | elapsed: 11.8min
[Parallel(n_jobs=-3)]: Done 2004 tasks      | elapsed: 15.4min
[Parallel(n_jobs=-3)]: Done 2548 tasks      | elapsed: 19.6min
[Parallel(n_jobs=-3)]: Done 3156 tasks      | elapsed: 24.2min
[Parallel(n_jobs=-3)]: Done 3828 tasks      | elapsed: 29.4min
[Parallel(n_jobs=-3)]: Done 4564 tasks      | elapsed: 35.0min
[Parallel(n_jobs=-3)]: Done 5364 tasks      | elapsed: 41.1min
[Parallel(n_jobs=-3)]: Done 6228 tasks      | elapsed: 47.7min
[Parallel(n_jobs=-3)]: Done 7156 tasks      | elapsed: 54.8min
[Parallel(n_jobs=-3)]: Done 8148 tasks      |

In [13]:
from functools import reduce

def calcualate_function_with_data(coefficient_list, variable_values):
    
    result = 0    
    for coefficient_value, coefficient_multipliers in zip(coefficient_list, list_of_monomial_identifiers):
        partial_results = [variable_value**int(coefficient_multiplier) for coefficient_multiplier, variable_value in zip(coefficient_multipliers, variable_values)]
        
        result += coefficient_value * reduce(lambda x, y: x*y, partial_results)

    return result, variable_values
 
def calculate_function_values_from_polynomial(true_value_test, evaluation_dataset):
    if isinstance(true_value_test, pd.DataFrame):
        true_value_test = true_value_test.values
        
    true_value_fv = []
    true_value_coeff = []
    
    for evaluation in evaluation_dataset:
        true_function_value, true_coeff = calcualate_function_with_data(true_value_test, evaluation)
       
        true_value_fv.append(true_function_value) 
        true_value_coeff.append(true_coeff)
        
    return [true_value_test, pd.DataFrame(np.array(true_value_coeff))], [true_value_test, pd.DataFrame(np.array(true_value_fv))]


In [14]:
result_list = []

polynomials_X_data_list = []
polynomials_y_data_list = []
    
chunks = max(interpretation_dataset_size//1000, 1)

for polynomials_list_df_chunk in tqdm(np.array_split(polynomials_list_df, chunks), total=chunks):
    parallel = Parallel(n_jobs=n_jobs, verbose=3, backend='loky')
    result_sublist = parallel(delayed(calculate_function_values_from_polynomial)(polynomial, x_values) for (_, polynomial), x_values in zip(polynomials_list_df_chunk.iterrows(), x_values_list))  
    result_list.extend(result_sublist)
    del parallel

polynomials_X_data_list = [result[0] for result in result_list]
polynomials_y_data_list = [result[1] for result in result_list]


[Parallel(n_jobs=-3)]: Using backend LokyBackend with 22 concurrent workers.
[Parallel(n_jobs=-3)]: Done  84 tasks      | elapsed:    7.4s
[Parallel(n_jobs=-3)]: Done 244 tasks      | elapsed:   19.0s
[Parallel(n_jobs=-3)]: Done 468 tasks      | elapsed:   42.3s
[Parallel(n_jobs=-3)]: Done 756 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-3)]: Done 1000 out of 1000 | elapsed:  1.5min finished
[Parallel(n_jobs=-3)]: Using backend LokyBackend with 22 concurrent workers.
[Parallel(n_jobs=-3)]: Done  84 tasks      | elapsed:    6.2s
[Parallel(n_jobs=-3)]: Done 244 tasks      | elapsed:   18.8s
[Parallel(n_jobs=-3)]: Done 468 tasks      | elapsed:   39.7s
[Parallel(n_jobs=-3)]: Done 756 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-3)]: Done 1000 out of 1000 | elapsed:  1.3min finished
[Parallel(n_jobs=-3)]: Using backend LokyBackend with 22 concurrent workers.
[Parallel(n_jobs=-3)]: Done  84 tasks      | elapsed:   11.2s
[Parallel(n_jobs=-3)]: Done 244 tasks      | elapsed:   24.7s
[Pa

[Parallel(n_jobs=-3)]: Done 756 tasks      | elapsed:   55.3s
[Parallel(n_jobs=-3)]: Done 1000 out of 1000 | elapsed:  1.3min finished
[Parallel(n_jobs=-3)]: Using backend LokyBackend with 22 concurrent workers.
[Parallel(n_jobs=-3)]: Done  84 tasks      | elapsed:    7.1s
[Parallel(n_jobs=-3)]: Done 244 tasks      | elapsed:   18.1s
[Parallel(n_jobs=-3)]: Done 468 tasks      | elapsed:   33.9s
[Parallel(n_jobs=-3)]: Done 756 tasks      | elapsed:   55.5s
[Parallel(n_jobs=-3)]: Done 1000 out of 1000 | elapsed:  1.2min finished
[Parallel(n_jobs=-3)]: Using backend LokyBackend with 22 concurrent workers.
[Parallel(n_jobs=-3)]: Done  84 tasks      | elapsed:    6.9s
[Parallel(n_jobs=-3)]: Done 244 tasks      | elapsed:   18.4s
[Parallel(n_jobs=-3)]: Done 468 tasks      | elapsed:   41.4s
[Parallel(n_jobs=-3)]: Done 756 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-3)]: Done 1000 out of 1000 | elapsed:  1.3min finished
[Parallel(n_jobs=-3)]: Using backend LokyBackend with 22 concurrent w

[Parallel(n_jobs=-3)]: Done 244 tasks      | elapsed:   15.8s
[Parallel(n_jobs=-3)]: Done 468 tasks      | elapsed:   29.9s
[Parallel(n_jobs=-3)]: Done 756 tasks      | elapsed:   48.0s
[Parallel(n_jobs=-3)]: Done 1000 out of 1000 | elapsed:  1.1min finished
[Parallel(n_jobs=-3)]: Using backend LokyBackend with 22 concurrent workers.
[Parallel(n_jobs=-3)]: Done  84 tasks      | elapsed:   11.3s
[Parallel(n_jobs=-3)]: Done 244 tasks      | elapsed:   21.3s
[Parallel(n_jobs=-3)]: Done 468 tasks      | elapsed:   35.4s
[Parallel(n_jobs=-3)]: Done 756 tasks      | elapsed:   53.6s
[Parallel(n_jobs=-3)]: Done 1000 out of 1000 | elapsed:  1.2min finished
[Parallel(n_jobs=-3)]: Using backend LokyBackend with 22 concurrent workers.
[Parallel(n_jobs=-3)]: Done  84 tasks      | elapsed:    5.6s
[Parallel(n_jobs=-3)]: Done 244 tasks      | elapsed:   15.8s
[Parallel(n_jobs=-3)]: Done 468 tasks      | elapsed:   30.1s
[Parallel(n_jobs=-3)]: Done 756 tasks      | elapsed:   48.3s
[Parallel(n_jobs=-

[Parallel(n_jobs=-3)]: Done  84 tasks      | elapsed:    5.6s
[Parallel(n_jobs=-3)]: Done 244 tasks      | elapsed:   15.8s
[Parallel(n_jobs=-3)]: Done 468 tasks      | elapsed:   29.9s
[Parallel(n_jobs=-3)]: Done 756 tasks      | elapsed:   48.1s
[Parallel(n_jobs=-3)]: Done 1000 out of 1000 | elapsed:  1.1min finished
[Parallel(n_jobs=-3)]: Using backend LokyBackend with 22 concurrent workers.
[Parallel(n_jobs=-3)]: Done  84 tasks      | elapsed:    5.6s
[Parallel(n_jobs=-3)]: Done 244 tasks      | elapsed:   15.8s
[Parallel(n_jobs=-3)]: Done 468 tasks      | elapsed:   30.0s
[Parallel(n_jobs=-3)]: Done 756 tasks      | elapsed:   48.3s
[Parallel(n_jobs=-3)]: Done 1000 out of 1000 | elapsed:  1.1min finished
[Parallel(n_jobs=-3)]: Using backend LokyBackend with 22 concurrent workers.
[Parallel(n_jobs=-3)]: Done  84 tasks      | elapsed:    5.7s
[Parallel(n_jobs=-3)]: Done 244 tasks      | elapsed:   15.8s
[Parallel(n_jobs=-3)]: Done 468 tasks      | elapsed:   30.0s
[Parallel(n_jobs=-

[Parallel(n_jobs=-3)]: Done 1000 out of 1000 | elapsed:  1.3min finished
[Parallel(n_jobs=-3)]: Using backend LokyBackend with 22 concurrent workers.
[Parallel(n_jobs=-3)]: Done  84 tasks      | elapsed:   16.5s
[Parallel(n_jobs=-3)]: Done 244 tasks      | elapsed:   29.2s
[Parallel(n_jobs=-3)]: Done 468 tasks      | elapsed:   45.7s
[Parallel(n_jobs=-3)]: Done 756 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-3)]: Done 1000 out of 1000 | elapsed:  1.4min finished
[Parallel(n_jobs=-3)]: Using backend LokyBackend with 22 concurrent workers.
[Parallel(n_jobs=-3)]: Done  84 tasks      | elapsed:    7.0s
[Parallel(n_jobs=-3)]: Done 244 tasks      | elapsed:   19.0s
[Parallel(n_jobs=-3)]: Done 468 tasks      | elapsed:   36.0s
[Parallel(n_jobs=-3)]: Done 756 tasks      | elapsed:   58.6s
[Parallel(n_jobs=-3)]: Done 1000 out of 1000 | elapsed:  1.3min finished
[Parallel(n_jobs=-3)]: Using backend LokyBackend with 22 concurrent workers.
[Parallel(n_jobs=-3)]: Done  84 tasks      | elapsed: 

[Parallel(n_jobs=-3)]: Done 1000 out of 1000 | elapsed:  1.3min finished


In [15]:
polynomials_X_data_list[0][0].head()

0000   -6.352
0001   -9.181
0002   -0.988
0003   -1.976
0010   -2.686
Name: 0, dtype: float64

In [16]:
polynomials_X_data_list[0][1].head()

,0,1,2,3
0,-0.33,-0.13,-0.95,0.01
1,-0.80,0.15,0.04,0.06
2,-0.96,0.09,-0.03,-0.05
3,-0.82,0.35,0.81,0.62
4,-0.50,0.05,0.44,0.69


In [17]:
polynomials_y_data_list[0][0].values

array([-6.352, -9.181, -0.988, -1.976, -2.686, -5.428, -6.642,  7.87 ,
       -7.152,  9.349,  3.825, -8.959, -9.024, -6.93 , -2.836, -2.377,
        6.559,  9.726, -9.131,  8.39 , -3.485,  7.856,  3.746, -2.777,
        4.719,  9.309, -0.885, -9.788, -4.769,  3.848,  1.149, -0.895,
       -4.906, -2.945,  1.029])

In [18]:
polynomials_y_data_list[0][0].head()

0000   -6.352
0001   -9.181
0002   -0.988
0003   -1.976
0010   -2.686
Name: 0, dtype: float64

In [19]:
polynomials_y_data_list[0][1].head()

,0
0,-7.379887
1,-4.025740
2,-1.664974
3,-24.713874
4,-20.389641


In [20]:
polynomials_X_data_list[0]

[0000   -6.352
 0001   -9.181
 0002   -0.988
 0003   -1.976
 0010   -2.686
 0011   -5.428
 0012   -6.642
 0020    7.870
 0021   -7.152
 0030    9.349
 0100    3.825
 0101   -8.959
 0102   -9.024
 0110   -6.930
 0111   -2.836
 0120   -2.377
 0200    6.559
 0201    9.726
 0210   -9.131
 0300    8.390
 1000   -3.485
 1001    7.856
 1002    3.746
 1010   -2.777
 1011    4.719
 1020    9.309
 1100   -0.885
 1101   -9.788
 1110   -4.769
 1200    3.848
 2000    1.149
 2001   -0.895
 2010   -4.906
 2100   -2.945
 3000    1.029
 Name: 0, dtype: float64,         0     1     2     3
 0   -0.33 -0.13 -0.95  0.01
 1   -0.80  0.15  0.04  0.06
 2   -0.96  0.09 -0.03 -0.05
 3   -0.82  0.35  0.81  0.62
 4   -0.50  0.05  0.44  0.69
 5   -0.06  0.30 -0.21  0.75
 6   -0.53  0.47 -0.36  0.12
 7    0.54  0.54  0.11 -0.69
 8   -0.53  0.29 -0.23 -0.22
 9    0.74 -0.87 -0.78  0.13
 10   0.95  0.40 -0.63 -0.57
 11   0.18 -0.10  0.43 -0.49
 12   0.70  0.64 -0.47 -0.23
 13   0.71  0.72 -0.86  0.87
 14  -0.88  0.3

In [21]:
polynomials_y_data_list[0]

[0000   -6.352
 0001   -9.181
 0002   -0.988
 0003   -1.976
 0010   -2.686
 0011   -5.428
 0012   -6.642
 0020    7.870
 0021   -7.152
 0030    9.349
 0100    3.825
 0101   -8.959
 0102   -9.024
 0110   -6.930
 0111   -2.836
 0120   -2.377
 0200    6.559
 0201    9.726
 0210   -9.131
 0300    8.390
 1000   -3.485
 1001    7.856
 1002    3.746
 1010   -2.777
 1011    4.719
 1020    9.309
 1100   -0.885
 1101   -9.788
 1110   -4.769
 1200    3.848
 2000    1.149
 2001   -0.895
 2010   -4.906
 2100   -2.945
 3000    1.029
 Name: 0, dtype: float64,              0
 0    -7.379887
 1    -4.025740
 2    -1.664974
 3   -24.713874
 4   -20.389641
 5   -12.780233
 6    -0.003142
 7     1.931115
 8     0.842917
 9     4.157199
 10   15.403501
 11   -1.331329
 12   11.531691
 13   10.133610
 14  -17.631749
 15  -11.644912
 16  -27.393539
 17    6.134937
 18   -6.210644
 19    6.574375
 20   -3.504210
 21   -2.308010
 22   -9.418847
 23    4.929815
 24    0.397516
 25   -8.333724
 26    5.604190
 2

In [22]:
path_polynomials = './data/saved_polynomial_lists/polynomials_sample' + str(interpretation_dataset_size) + '_variables_' + str(n) +  '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step)  + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '.csv'
polynomials_list_df.to_csv(path_polynomials, index=False)

path_X_data = './data/saved_polynomial_lists/X_sample' + str(interpretation_dataset_size) + '_train_' + str(lambda_dataset_size) + '_variables_' + str(n) + '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step) + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '_xstep_' + str(x_step) + '_xmin_' + str(x_min) + '_xmax_' + str(x_max) + training_string + '.pkl'
with open(path_X_data, 'wb') as f:
    pickle.dump(polynomials_X_data_list, f)#, protocol=2)
    
path_y_data = './data/saved_polynomial_lists/y_sample' + str(interpretation_dataset_size) + '_train_' + str(lambda_dataset_size) + '_variables_' + str(n) + '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step) + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '_xstep_' + str(x_step) + '_xmin_' + str(x_min) + '_xmax_' + str(x_max) + training_string + '.pkl'
with open(path_y_data, 'wb') as f:
    pickle.dump(polynomials_y_data_list, f)#, protocol=2)
